<img src="../img/Signet_FNW_1.svg" alt="OVGU_FNW_Logo" width="300" align="right">

# Lecture Tutorial 1F: FT Winding Machine

Idea based on 3Blue1Brown's video ["But what is the Fourier Transform? A visual introduction."](https://www.youtube.com/watch?v=spUNpyF58BY).

There are other implementations:
- [https://prajwalsouza.github.io/Experiments/Fourier-Transform-Visualization.html](https://prajwalsouza.github.io/Experiments/Fourier-Transform-Visualization.html) (interactive)
- [https://github.com/thatSaneKid/fourier/blob/master/Fourier%20Transform%20-%20A%20Visual%20Introduction.ipynb](https://github.com/thatSaneKid/fourier/blob/master/Fourier%20Transform%20-%20A%20Visual%20Introduction.ipynb) (notebook)



In [ ]:
# FT-Winding-Machine Idea from 3Blue1Brown's video "But what is the Fourier Transform? A visual introduction." (https://www.youtube.com/watch?v=spUNpyF58BY) 
# Idea extended by Hendrik Mattern
# Coded mainly by AI, and tweaked by Hendrik Mattern

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import ipywidgets as widgets

# ---- Constants & Global State ----
DURATION = 4.0          # Seconds of signal
SAMPLING_RATE = 500     # Hz
N_POINTS = int(DURATION * SAMPLING_RATE)
TIME = np.linspace(0, DURATION, N_POINTS)
FIG_SIZE = (8, 4)

MEASUREMENT_HISTORY = {
    "params": None,  
    "mode": None,    
    "freqs": [],     
    "vals": []       
}

def generate_signal(t, amps, freqs):
    """Summation of 5 sine waves. Returns total signal and individual components."""
    total_signal = np.zeros_like(t)
    components = []
    for A, f in zip(amps, freqs):
        # CHANGED TO SIN FOR BETTER WAVE SHAPE ALIGNMENT
        comp = A * np.sin(2 * np.pi * f * t) 
        components.append(comp)
        total_signal += comp
    return total_signal, components

def get_center_of_mass(signal, t, winding_freq):
    """Wraps signal around a circle and calculates Center of Mass."""
    winders = np.exp(-2j * np.pi * winding_freq * t)
    wound_signal = signal * winders
    center_of_mass = np.mean(wound_signal)
    return wound_signal, center_of_mass

def draw_fourier_machine(f_test, show_components, mode, A1, f1, A2, f2, A3, f3, A4, f4, A5, f5):
    current_params = (A1, f1, A2, f2, A3, f3, A4, f4, A5, f5)
    if MEASUREMENT_HISTORY["params"] != current_params or MEASUREMENT_HISTORY["mode"] != mode:
        MEASUREMENT_HISTORY["params"] = current_params
        MEASUREMENT_HISTORY["mode"] = mode
        MEASUREMENT_HISTORY["freqs"], MEASUREMENT_HISTORY["vals"] = [], []
        
    amps, freqs = [A1, A2, A3, A4, A5], [f1, f2, f3, f4, f5]
    g_t, components = generate_signal(TIME, amps, freqs)
    wound_signal, com = get_center_of_mass(g_t, TIME, f_test)
    
    if mode == "CoM Magnitude": current_val = np.abs(com)
    elif mode == "X Component (Real)": current_val = com.real
    else: current_val = com.imag
    
    if not MEASUREMENT_HISTORY["freqs"] or round(f_test, 3) != MEASUREMENT_HISTORY["freqs"][-1]:
        MEASUREMENT_HISTORY["freqs"].append(round(f_test, 3))
        MEASUREMENT_HISTORY["vals"].append(current_val)

    fig = plt.figure(figsize=FIG_SIZE)
    gs = GridSpec(2, 2, height_ratios=[1, 1], width_ratios=[1, 1], figure=fig)
    
    # -- 1. Time Domain --
    ax_time = fig.add_subplot(gs[0, 0])
    if show_components:
        for comp in components:
            if np.max(np.abs(comp)) > 0.01: ax_time.plot(TIME, comp, color='gray', linestyle='--', alpha=0.3)
    ax_time.plot(TIME, g_t, color='#f0c629', linewidth=2)
    ax_time.set_title("1. Input Signal g(t)", loc='left', fontsize=11, fontweight='bold')
    ax_time.set_xlim(0, 2)
    ax_time.grid(True, alpha=0.3)
    
    # -- 2. Winding Machine --
    ax_wind = fig.add_subplot(gs[:, 1])
    ax_wind.plot(wound_signal.real, wound_signal.imag, color='#f57878', alpha=0.4, linewidth=0.8)
    ax_wind.plot([0, com.real], [0, 0], color='blue', linewidth=2, alpha=0.7)
    ax_wind.plot([com.real, com.real], [0, com.imag], color='green', linewidth=2, alpha=0.7)
    ax_wind.plot([0, com.real], [0, com.imag], color='red', linestyle='-', linewidth=1.5)
    ax_wind.scatter([com.real], [com.imag], color='red', s=80, zorder=10, edgecolors='white')
    ax_wind.set_title(f"2. Winding Machine (f = {f_test:.2f} Hz)", loc='left', fontsize=11, fontweight='bold')
    limit = max(np.max(np.abs(g_t)), 1.0) * 1.2
    ax_wind.set_xlim(-limit, limit); ax_wind.set_ylim(-limit, limit); ax_wind.set_aspect('equal')

    # -- 3. Spectrum History --
    ax_freq = fig.add_subplot(gs[1, 0])
    if MEASUREMENT_HISTORY["freqs"]: ax_freq.scatter(MEASUREMENT_HISTORY["freqs"], MEASUREMENT_HISTORY["vals"], color='#1b9bd7', s=15, alpha=0.6)
    ax_freq.scatter([f_test], [current_val], color='red', s=80, zorder=10, edgecolors='white')
    ax_freq.set_title(f"3. {mode} Sweep", loc='left', fontsize=11, fontweight='bold')
    ax_freq.set_xlim(0, 10); ax_freq.grid(True, alpha=0.3)
    
    plt.tight_layout(); plt.show()

# ---- UI Construction ----
w_freq_slider = widgets.FloatSlider(value=0.0, min=0.0, max=10.0, step=0.02, description="<b>Winding Freq</b>", layout=widgets.Layout(width='98%'))
chk_components = widgets.Checkbox(value=False, description="Show Components")
mode_dropdown = widgets.Dropdown(options=["CoM Magnitude", "X Component (Real)", "Y Component (Imaginary)"], value="CoM Magnitude", description="<b>Plot Mode:</b>")
chk_rect = widgets.Checkbox(value=False, description="Preset: Rect")
chk_saw = widgets.Checkbox(value=False, description="Preset: Saw")

def make_input_row(i, def_A, def_f):
    return widgets.HBox([widgets.FloatText(value=def_A, description=f'A{i}', step=0.1, layout=widgets.Layout(width='150px')),
                        widgets.FloatText(value=def_f, description=f'f{i}', step=0.1, layout=widgets.Layout(width='150px'))])

rows = [make_input_row(i+1, 0.0, float(i+1)) for i in range(5)]
rows[0].children[0].value = 1.0

def apply_rect_preset(change):
    if change['new']:
        chk_saw.value = False
        preset_vals = [(1.0, 1.0), (0.33, 3.0), (0.2, 5.0), (0.14, 7.0), (0.11, 9.0)]
        for i, (a, f) in enumerate(preset_vals):
            rows[i].children[0].value, rows[i].children[1].value = a, f

def apply_saw_preset(change):
    if change['new']:
        chk_rect.value = False
        preset_vals = [(1.0, 1.0), (0.5, 2.0), (0.33, 3.0), (0.25, 4.0), (0.2, 5.0)]
        for i, (a, f) in enumerate(preset_vals):
            rows[i].children[0].value, rows[i].children[1].value = a, f

chk_rect.observe(apply_rect_preset, names='value')
chk_saw.observe(apply_saw_preset, names='value')

ui = widgets.VBox([w_freq_slider, widgets.HBox([mode_dropdown, chk_components, chk_rect, chk_saw])] + rows)
out = widgets.interactive_output(draw_fourier_machine, {'f_test': w_freq_slider, 'show_components': chk_components, 'mode': mode_dropdown,
    'A1': rows[0].children[0], 'f1': rows[0].children[1], 'A2': rows[1].children[0], 'f2': rows[1].children[1],
    'A3': rows[2].children[0], 'f3': rows[2].children[1], 'A4': rows[3].children[0], 'f4': rows[3].children[1],
    'A5': rows[4].children[0], 'f5': rows[4].children[1]})

display(ui, out)

Output()